In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [6]:
train_df=pd.read_csv('/content/drive/MyDrive/CANIDS/Data/normal_run_data.csv')
train_df['label']='R'
colnames=['Timestamp', 'ID', 'DLC', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8','label'] 
test_df=pd.read_csv('/content/drive/MyDrive/CANIDS/Data/DoS_dataset.csv',names=colnames)

In [7]:
train_df.head()

,Timestamp,ID,DLC,D1,D2,D3,D4,D5,D6,D7,D8,label
0,1.479121e+09,0350,8,05,28,84,66,6d,00,00,a2,R
1,1.479121e+09,02c0,8,14,00,00,00,00,00,00,00,R
2,1.479121e+09,0430,8,00,00,00,00,00,00,00,00,R
3,1.479121e+09,04b1,8,00,00,00,00,00,00,00,00,R
4,1.479121e+09,01f1,8,00,00,00,00,00,00,00,00,R


In [8]:
test_df.head()

,Timestamp,ID,DLC,D1,D2,D3,D4,D5,D6,D7,D8,label
0,1.478198e+09,0316,8,05,21,68,09,21,21,00,6f,R
1,1.478198e+09,018f,8,fe,5b,00,00,00,3c,00,00,R
2,1.478198e+09,0260,8,19,21,22,30,08,8e,6d,3a,R
3,1.478198e+09,02a0,8,64,00,9a,1d,97,02,bd,00,R
4,1.478198e+09,0329,8,40,bb,7f,14,11,20,00,14,R


In [9]:
def h2i(s):
    return int(s,16)

In [10]:
def s2i(s):
    if s=='R':
      return 0
    else:
      return 1

In [11]:
train_df['ID']=train_df['ID'].apply(h2i)
train_df['label']=train_df['label'].apply(s2i)
test_df['ID']=test_df['ID'].apply(h2i)
test_df['label']=test_df['label'].apply(s2i)


In [12]:
train_df.head()

,Timestamp,ID,DLC,D1,D2,D3,D4,D5,D6,D7,D8,label
0,1.479121e+09,848,8,05,28,84,66,6d,00,00,a2,0
1,1.479121e+09,704,8,14,00,00,00,00,00,00,00,0
2,1.479121e+09,1072,8,00,00,00,00,00,00,00,00,0
3,1.479121e+09,1201,8,00,00,00,00,00,00,00,00,0
4,1.479121e+09,497,8,00,00,00,00,00,00,00,00,0


In [13]:
test_df.head()

,Timestamp,ID,DLC,D1,D2,D3,D4,D5,D6,D7,D8,label
0,1.478198e+09,790,8,05,21,68,09,21,21,00,6f,0
1,1.478198e+09,399,8,fe,5b,00,00,00,3c,00,00,0
2,1.478198e+09,608,8,19,21,22,30,08,8e,6d,3a,0
3,1.478198e+09,672,8,64,00,9a,1d,97,02,bd,00,0
4,1.478198e+09,809,8,40,bb,7f,14,11,20,00,14,0


In [14]:
X_train=train_df.iloc[:,0]
y_train=train_df.iloc[:,-1]

In [15]:
X_train.head()

0    1.479121e+09
1    1.479121e+09
2    1.479121e+09
3    1.479121e+09
4    1.479121e+09
Name: Timestamp, dtype: float64

In [16]:
y_train.head()

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

In [42]:
X_test=test_df.iloc[:,0]
y_test=test_df.iloc[:,-1]

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [36]:
regressor=Sequential()

In [37]:
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(1,1)))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

In [38]:
regressor.add(Dense(units=1))

In [21]:
def my_loss(y_true,y_pred):
  if y_true==y_pred:
    return 0
  else:
    return 1

In [39]:
regressor.compile(optimizer='adam',loss=tf.losses.MeanSquaredError())

In [40]:
regressor.fit(X_train,y_train,epochs=1,batch_size=32)

30899/30899 [==============================] - 210s 7ms/step - loss: 1.3646e-08


In [ ]:
print(regressor.predict_on_batch(tf.expand_dims(X_test,0),axis=1))